In [1]:
# Dalvin Colbert
## D603 Machine Learning
### RFN1 Task 1: Classification Data Mining Models

In [2]:
# Import appropriate modules
import random
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import seaborn as sns
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
from sklearn.datasets import make_regression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from scipy import stats
from scipy.stats import randint
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Import Data set
df = pd.DataFrame(pd.read_csv("C:\\Users\\dalvi\\OneDrive - Western Governors University\\MSDADS\\D603 - Machine Learning\\Writeup\\Task 1\\churn_clean.csv", na_values=['NA', 'N/A'], keep_default_na=False))
num_rows, num_columns = df.shape

# Describe the general characteristcs of the inital dataset (e.g., rows, columns)
print(f"The dataset contains {num_rows} rows and {num_columns} columns.")
print(f"The variables are {df.columns}")

In [4]:
# Copy the dataset for integrity
df_copy = df.copy()

In [ ]:
# Examine first five rows of dataframe
df.head()

In [ ]:
df.columns

In [7]:
df = df.drop(['CaseOrder', 'Customer_id', 'Interaction', 'UID', 'City', 'State', 'County', 'Zip','Lat', 'Lng', 'Population', 'Area', 'TimeZone', 'Item1','Item2','Item3','Item4','Item5','Item6','Item7','Item8', 'Job'], axis=1)


In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"The categorical variables are {categorical_columns}")
print(f"The numerical variables are {numerical_columns}")


In [ ]:
# Identify duplicate rows
duplicates = df.duplicated().sum()
print(f"Number of duplicate entries: {duplicates}")
print("\n\n")

In [ ]:
# Missing values
total_missing_values = df.isna().sum().sum()
print('Total missing values: ', total_missing_values)
print(df.isna().sum())

In [ ]:
categorical_columns

In [14]:
# List of columns to encode
columns_to_labelencode = ['Churn', 'Techie', 'Port_modem', 'Tablet', 'Phone', 'Multiple', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
columns_to_onehotencode = ['Gender','Marital', 'InternetService', 'PaymentMethod', 'Contract']

# Apply label encoding to each specified column
for column in columns_to_labelencode:
    df[f'{column}_encoded'] = LabelEncoder().fit_transform(df[column])



In [ ]:
df = df.drop(columns=columns_to_labelencode)
df.columns

In [16]:
df_encoded = pd.get_dummies(df, columns=columns_to_onehotencode, dtype=int)


In [ ]:
df_encoded.columns

In [ ]:
df_encoded.head()

In [ ]:
num_rows_enc, num_columns_enc = df_encoded.shape

# Describe the general characteristcs of the inital dataset (e.g., rows, columns)
print(f"The dataset contains {num_rows_enc} rows and {num_columns_enc} columns.")
print(f"The variables are {df.columns}")

In [20]:
df_encoded.to_csv('C:\\Users\\dalvi\\OneDrive - Western Governors University\\MSDADS\\D603 - Machine Learning\\Writeup\\Task 1\\df_encoded_CLEAN.csv')


In [21]:
# Train validate test (60 / 20 / 20 split)
train_df, temp_df = train_test_split(df_encoded, test_size=0.4, random_state=8)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=8)



In [22]:
# Assuming X_train, X_test, y_train, y_test are already defined
X_train = train_df.drop(columns='Churn_encoded')
y_train = train_df['Churn_encoded']

X_test = test_df.drop(columns='Churn_encoded')
y_test = test_df['Churn_encoded']

X_val = val_df.drop(columns='Churn_encoded')
y_val = val_df['Churn_encoded']
# Train, val, test splits complete 

In [ ]:
## Initial Model Creation
# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

## Predictions from initial model creation
# Make predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
auc_roc = roc_auc_score(y_test, y_pred_proba)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [24]:
#train_df.to_csv('train_dataset.csv', index=False)
#val_df.to_csv('val_dataset.csv', index=False)
#test_df.to_csv('test_dataset.csv', index=False)
#X_train.to_csv('X_train.csv', index=False)
#y_train.to_csv('y_train.csv', index=False)
#X_test.to_csv('X_test.csv', index=False)
#y_test.to_csv('y_test.csv', index=False)
#X_val.to_csv('X_val.csv', index=False)
#y_val.to_csv('y_val.csv', index=False)

In [ ]:
# Hyperparameter analysis
# Tuning
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_val, y_val)

best_model = grid_search.best_estimator_
feature_importance = best_model.feature_importances_

predictions = best_model.predict(X_test)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)
print("Best model:", best_model)



In [ ]:
# Optimized Model Predictions 
# Make predictions
predictions = best_model.predict(X_test)
predictions_proba = best_model.predict_proba(X_test)[:, 1]


# Calculate metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')
auc_roc = roc_auc_score(y_test, predictions_proba)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

# Generate confusion matrix
cm = confusion_matrix(y_test, predictions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
